<a href="https://colab.research.google.com/github/Avina20/MCQ-generator--NLP-project/blob/main/MCQ_Question_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import random
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import os
nlp = spacy.load('en_core_web_sm')

In [2]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.test.utils import datapath
from gensim.test.utils import common_texts, get_tmpfile

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
fname = get_tmpfile('/content/drive/My Drive/MCQ generator/glove.6B.50d.txt')

In [6]:
model = Word2Vec.load_word2vec_format

In [ ]:
from google.colab import files
files.upload()

Saving word2vec.txt to word2vec.txt


In [7]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [10]:
glove2word2vec(fname, 'word2vec.txt')

(400000, 50)

In [11]:
model = KeyedVectors.load_word2vec_format('word2vec.txt')

In [12]:
model.save('incorrect_answer_model.pkl')

In [13]:
model.most_similar(positive=['oxygen'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('hydrogen', 0.808455765247345),
 ('liquid', 0.7736512422561646),
 ('nitrogen', 0.7624309659004211),
 ('chlorine', 0.7590859532356262),
 ('helium', 0.752963125705719),
 ('ammonia', 0.7375549077987671),
 ('methane', 0.731142520904541),
 ('blood', 0.722781777381897),
 ('nutrients', 0.7218273282051086),
 ('fluids', 0.7213950157165527)]

#TEXT SUMMARIZATION

In [14]:
def text_summarizer(text):
  list_sent = []
  doc = nlp(text)
  for sent in doc.sents:
    list_sent.append(str(sent))
  num_sent = len(list_sent)
  list_embed = []
  for sent in list_sent:
    doc_sent = nlp(sent)
    list_embed.append(doc_sent.vector.reshape(1,96))
  similarity = np.zeros([num_sent, num_sent])
  for i in range(num_sent):
    for j in range(num_sent):
      similarity[i][j] = cosine_similarity(list_embed[i], list_embed[j])[0][0]
  nx_graph = nx.from_numpy_array(similarity)
  scores = nx.pagerank(nx_graph)
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(list_sent)), reverse=True)
  sentences = []
  for score, sent in ranked_sentences:
    sentences.append(sent)
  return sentences

#KEYWORD EXTRACTION

In [15]:
def ireplace(old, new, text):
    index_l = text.lower().index(old.lower())
    return text[:index_l] + new + text[index_l + len(old):]

In [29]:
def return_question_with_answer(text):
  try:
    text = str(text).replace('\n',' ')
    doc = nlp(text)
    if(len(doc.ents)>0):
      ans = str(random.choice(doc.ents))
      ques = ireplace(ans, '_______',text)
      return ques, ans
    else:
      return -1, -1
  except:
    return -1,-1

#DISTRACTOR GENERATOR

In [17]:
def distractor_generator(word):
  word = word.lower()
  if(word in model.wv.vocab):
    similar = model.most_similar(positive=[word],topn=3)
    ans = []
    for tup in similar:
      ans.append(tup[0])
    return ans
  else:
    return []

In [18]:
def print_questions_with_answers(summary):
  ctr = 0
  for i in range(len(summary)):
    question, answer = return_question_with_answer(summary[i])
    if(question!=-1):
      choices = distractor_generator(answer)
      if(len(choices)>0):
        print('Q',(ctr+1),': ',question)
        choices.append(answer)
        random.shuffle(choices)
        print("A.", choices[0], "\tB:", choices[1],"\tC:", choices[2], "\tD:", choices[3])
        print('Answer: ', answer)
        ctr = ctr+1

In [ ]:
carbon_text = """    Carbon gets its name from the Latin word carbo, which means "coal." Diamonds and graphite are among the hardest and softest natural materials known, respectively. The only difference between the two is their crystal structure.Carbon makes up 0.032 percent of the Earth's lithosphere (crust and outer mantle) by weight, according to the Encyclopedia of Earth. A rough estimate of the weight of the lithosphere by La Salle University geologist David Smith is 300,000,000,000,000,000,000,000 (or 3*10^23) pounds, making the approximate weight of carbon in the lithosphere 10,560,000,000,000,000,000,000 (or 1.056*10^22) pounds. Carbon dioxide (a carbon atom plus two oxygen atoms) makes up about 0.04 percent of Earth's atmosphere, according to the National Oceanic & Atmospheric Administration (NOAA) — an increase over pre-industrial times, because of the burning of fossil fuels. Carbon monoxide (a carbon atom plus one oxygen atom) is an odorless gas produced from the burning of fossil fuels. Carbon monoxide kills by binding to hemoglobin, the oxygen-carrying compound in the blood. Carbon monoxide bonds to hemoglobin 210 times more strongly than oxygen binds to hemoglobin, effectively crowding out oxygen and suffocating the tissues, according to a 2001 paper in the Journal of the Royal Society of Medicine. Diamond, the flashiest version of carbon, is formed under great pressure deep in the Earth's crust. The largest gem-quality diamond ever found was the Cullinan diamond, which was discovered in 1905, according to the Royal Collection Trust. The uncut diamond was 3,106.75 carats. The largest gem cut from the stone, at 530.2 carats, is one of the Crown Jewels of the United Kingdom and is known as the Great Star of Africa. The tattoos of Ötzi the Iceman, a 5,300-year-old corpse found frozen in the Alps, were inked from carbon, according to a 2009 study in the Journal of Archaeological Science. Small incisions in the skin were made, and charcoal rubbed in, perhaps as part of an acupuncture treatment."""

In [ ]:
carbon_text

'    Carbon gets its name from the Latin word carbo, which means "coal." Diamonds and graphite are among the hardest and softest natural materials known, respectively. The only difference between the two is their crystal structure.Carbon makes up 0.032 percent of the Earth\'s lithosphere (crust and outer mantle) by weight, according to the Encyclopedia of Earth. A rough estimate of the weight of the lithosphere by La Salle University geologist David Smith is 300,000,000,000,000,000,000,000 (or 3*10^23) pounds, making the approximate weight of carbon in the lithosphere 10,560,000,000,000,000,000,000 (or 1.056*10^22) pounds. Carbon dioxide (a carbon atom plus two oxygen atoms) makes up about 0.04 percent of Earth\'s atmosphere, according to the National Oceanic & Atmospheric Administration (NOAA) — an increase over pre-industrial times, because of the burning of fossil fuels. Carbon monoxide (a carbon atom plus one oxygen atom) is an odorless gas produced from the burning of fossil fuels

In [19]:
text = """The Third Battle of Panipat took place on 14 January 1761 at Panipat, about 97 km (60 miles) north of Delhi, between the Maratha Empire and the invading Afghan army of the King of Afghans, Ahmad Shah Abdali, supported by three Indian allies — the Rohilla Najib-ud-daulah, Afghans of the Doab region, and Shuja-ud-Daula, the Nawab of Awadh. The Maratha army was led by Sadashivrao Bhau who was third in authority after the Chhatrapati (Maratha King) and the Peshwa (Maratha Prime Minister). The main Maratha army was stationed in Deccan with the Peshwa. Militarily, the battle pitted the artillery and cavalry of the Marathas against the heavy cavalry and mounted artillery (zamburak and jezail) of the Afghans and Rohillas led by Abdali and Najib-ud-Daulah, both ethnic Afghans. The battle is considered one of the largest and most eventful fought in the 18th century,[10] and it has perhaps the largest number of fatalities in a single day reported in a classic formation battle between two armies. The specific site of the battle itself is disputed by historians, but most consider it to have occurred somewhere near modern-day Kaalaa Aamb and Sanauli Road. The battle lasted for several days and involved over 125,000 troops. Protracted skirmishes occurred, with losses and gains on both sides. The forces led by Ahmad Shah Durrani came out victorious after destroying several Maratha flanks. The extent of the losses on both sides is heavily disputed by historians, but it is believed that between 60,000–70,000 were killed in fighting, while the numbers of injured and prisoners taken vary considerably. According to the single best eyewitness chronicle—the bakhar by Shuja-ud-Daulah's Diwan Kashi Raj—about 40,000 Maratha prisoners were slaughtered in cold blood the day after the battle.[9] Grant Duff includes an interview of a survivor of these massacres in his History of the Marathas and generally corroborates this number. Shejwalkar, whose monograph Panipat 1761 is often regarded as the single best secondary source on the battle, says that 'not less than 100,000 Marathas (soldiers and non-combatants) perished during and after the battle.'"""

In [20]:
summary = text_summarizer(text)

In [22]:
return_question_with_answer(summary)

('[\'The Third Battle of Panipat took place on 14 January 1761 at Panipat, _______ (60 miles) north of Delhi, between the Maratha Empire and the invading Afghan army of the King of Afghans,\', "According to the single best eyewitness chronicle—the bakhar by Shuja-ud-Daulah\'s Diwan Kashi Raj—about 40,000 Maratha prisoners were slaughtered in cold blood the day after the battle.[9]", \'The main Maratha army was stationed in Deccan with the Peshwa.\', \'The specific site of the battle itself is disputed by historians, but most consider it to have occurred somewhere near modern-day Kaalaa Aamb and Sanauli Road.\', "Shejwalkar, whose monograph Panipat 1761 is often regarded as the single best secondary source on the battle, says that \'not less than 100,000 Marathas (soldiers and non-combatants) perished during and after the battle.\'", \'Militarily, the battle pitted the artillery and cavalry of the Marathas against the heavy cavalry and mounted artillery (zamburak and jezail) of the Afgh

In [23]:
print_questions_with_answers(summary)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Q 1 :  The main _______ army was stationed in Deccan with the Peshwa.
A. seljuq 	B: seleucid 	C: seljuk 	D: Maratha
Answer:  Maratha
Q 2 :  The _______ army was led by Sadashivrao Bhau who was third in authority after the Chhatrapati (Maratha King) and the Peshwa (Maratha Prime Minister).
A. Maratha 	B: seljuk 	C: seljuq 	D: seleucid
Answer:  Maratha
Q 3 :  The forces led by Ahmad Shah Durrani came out victorious after destroying several _______ flanks.
A. seljuq 	B: seleucid 	C: seljuk 	D: Maratha
Answer:  Maratha
No entities


In [ ]:
return_questions_with_answers(summary)

Q 1 :  Shejwalkar, whose monograph Panipat _______ is often regarded as the single best secondary source on the battle, says that 'not less than 100,000 Marathas (soldiers and non-combatants) perished during and after the battle.'
Answer:  1761
Q 2 :  According to the single best eyewitness chronicle—the bakhar by Shuja-ud-Daulah's Diwan Kashi Raj—about 40,000 _______ prisoners were slaughtered in cold blood the day after the battle.[9]
Answer:  Maratha
Q 3 :  Militarily, the battle pitted the artillery and cavalry of the Marathas against the heavy cavalry and mounted artillery (zamburak and jezail) of the _______ and Rohillas led by Abdali and Najib-ud-Daulah, both ethnic Afghans.
Answer:  Afghans
Q 4 :  The battle is considered one of the largest and most eventful fought in the 18th century,[10] and it has perhaps the largest number of fatalities in _______ reported in a classic formation battle between two armies.
Answer:  a single day
Q 5 :  The specific site of the battle itself i

In [ ]:
text = """The Taj Mahal is an ivory-white marble mausoleum on the southern bank of the river Yamuna in the Indian city of Agra. It was commissioned in 1632 by the Mughal emperor Shah Jahan (reigned from 1628 to 1658) to house the tomb of his favourite wife, Mumtaz Mahal; it also houses the tomb of Shah Jahan himself. The tomb is the centrepiece of a 17-hectare (42-acre) complex, which includes a mosque and a guest house, and is set in formal gardens bounded on three sides by a crenellated wall. Construction of the mausoleum was essentially completed in 1643, but work continued on other phases of the project for another 10 years. The Taj Mahal complex is believed to have been completed in its entirety in 1653 at a cost estimated at the time to be around 32 million rupees, which in 2020 would be approximately 70 billion rupees (about U.S. $916 million). The construction project employed some 20,000 artisans under the guidance of a board of architects led by the court architect to the emperor, Ustad Ahmad Lahauri. The Taj Mahal was designated as a UNESCO World Heritage Site in 1983 for being "the jewel of Muslim art in India and one of the universally admired masterpieces of the world's heritage". It is regarded by many as the best example of Mughal architecture and a symbol of India's rich history. The Taj Mahal attracts 7–8 million visitors a year and in 2007, it was declared a winner of the New 7 Wonders of the World (2000–2007) initiative."""

In [ ]:
summary = text_summarizer(text)

In [ ]:
summary

['The Taj Mahal was designated as a UNESCO World Heritage Site in 1983 for being "the jewel of Muslim art in India and one of the universally admired masterpieces of the world\'s heritage".',
 'The construction project employed some 20,000 artisans under the guidance of a board of architects led by the court architect to the emperor, Ustad Ahmad Lahauri.',
 'The tomb is the centrepiece of a 17-hectare (42-acre) complex, which includes a mosque and a guest house, and is set in formal gardens bounded on three sides by a crenellated wall.',
 'It was commissioned in 1632 by the Mughal emperor Shah Jahan (reigned from 1628 to 1658) to house the tomb of his favourite wife, Mumtaz Mahal; it also houses the tomb of Shah Jahan himself.',
 'The Taj Mahal attracts 7–8 million visitors a year and in 2007, it was declared a winner of the New 7 Wonders of the World (2000–2007) initiative.',
 'Construction of the mausoleum was essentially completed in 1643, but work continued on other phases of the p

In [ ]:
return_question_with_answer(summary[0])

('The Taj Mahal was designated as a UNESCO World Heritage Site in 1983 for being "the jewel of _______ art in India and one of the universally admired masterpieces of the world\'s heritage".',
 'Muslim')

In [ ]:
print_questions_with_answers(summary)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Q 1 :  The Taj Mahal was designated as a UNESCO World Heritage Site in 1983 for being "the jewel of _______ art in India and one of the universally admired masterpieces of the world's heritage".
A. islamic 	B: moslem 	C: Muslim 	D: muslims
Answer:  Muslim
Q 2 :  The tomb is the centrepiece of a 17-hectare (42-acre) complex, which includes a mosque and a guest house, and is set in formal gardens bounded on _______ sides by a crenellated wall.
A. five 	B: four 	C: three 	D: two
Answer:  three
Q 3 :  It was commissioned in 1632 by the _______ emperor Shah Jahan (reigned from 1628 to 1658) to house the tomb of his favourite wife, Mumtaz Mahal; it also houses the tomb of Shah Jahan himself.
A. Mughal 	B: aurangzeb 	C: chola 	D: qajar
Answer:  Mughal
Q 4 :  Construction of the mausoleum was essentially completed in _______, but work continued on other phases of the project for another 10 years.
A. 1646 	B: 1641 	C: 1642 	D: 1643
Answer:  1643
Q 5 :  It is regarded by many as the best example

In [24]:
text = """Carbon is a chemical element with the symbol C and atomic number 6. It is nonmetallic and tetravalent—making four electrons available to form covalent chemical bonds. It belongs to group 14 of the periodic table. Carbon makes up only about 0.025 percent of Earth’s crust. Three isotopes occur naturally, 12C and 13C being stable, while 14C is a radionuclide, decaying with a half-life of about 5,730 years. Carbon is one of the few elements known since antiquity. Carbon is the 15th most abundant element in the Earth's crust, and the fourth most abundant element in the universe by mass after hydrogen, helium, and oxygen. Carbon's abundance, its unique diversity of organic compounds, and its unusual ability to form polymers at the temperatures commonly encountered on Earth enables this element to serve as a common element of all known life. It is the second most abundant element in the human body by mass (about 18.5%) after oxygen. The atoms of carbon can bond together in diverse ways, resulting in various allotropes of carbon. The best known allotropes are graphite, diamond, and buckminsterfullerene. The physical properties of carbon vary widely with the allotropic form. For example, graphite is opaque and black while diamond is highly transparent. Graphite is soft enough to form a streak on paper (hence its name, from the Greek verb "γράφειν" which means "to write"), while diamond is the hardest naturally occurring material known. Graphite is a good electrical conductor while diamond has a low electrical conductivity. Under normal conditions, diamond, carbon nanotubes, and graphene have the highest thermal conductivities of all known materials. All carbon allotropes are solids under normal conditions, with graphite being the most thermodynamically stable form at standard temperature and pressure. They are chemically resistant and require high temperature to react even with oxygen. The most common oxidation state of carbon in inorganic compounds is +4, while +2 is found in carbon monoxide and transition metal carbonyl complexes. The largest sources of inorganic carbon are limestones, dolomites and carbon dioxide, but significant quantities occur in organic deposits of coal, peat, oil, and methane clathrates. Carbon forms a vast number of compounds, more than any other element, with almost ten million compounds described to date, and yet that number is but a fraction of the number of theoretically possible compounds under standard conditions. For this reason, carbon has often been referred to as the "king of the elements"."""

In [25]:
summary=text_summarizer(text)

In [30]:
print_questions_with_answers(summary)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Q 1 :  Carbon's abundance, its unique diversity of organic compounds, and its unusual ability to form polymers at the temperatures commonly encountered on _______ enables this element to serve as a common element of all known life.
A. mars 	B: Earth 	C: planet 	D: space
Answer:  Earth
Q 2 :  Three isotopes occur naturally, _______ and 13C being stable, while 14C is a radionuclide, decaying with a half-life of about 5,730 years.
A. 5700 	B: 3700 	C: 14c 	D: 12C
Answer:  12C
Q 3 :  It is the _______ most abundant element in the human body by mass (about 18.5%) after oxygen.
A. fourth 	B: third 	C: fifth 	D: second
Answer:  second
Q 4 :  Carbon is a chemical element with the symbol C and atomic number _______.
A. 8 	B: 6 	C: 4 	D: 5
Answer:  6
Q 5 :  Carbon is the 15th most abundant element in the _______'s crust, and the fourth most abundant element in the universe by mass after hydrogen, helium, and oxygen.
A. Earth 	B: space 	C: planet 	D: mars
Answer:  Earth
Q 6 :  Carbon is _______ o

#Bert embedding

In [ ]:
!pip install bert-embedding

     |████████████████████████████████| 215kB 3.9MB/s 
     |████████████████████████████████| 29.6MB 144kB/s 
     |████████████████████████████████| 13.8MB 324kB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-cp36-none-any.whl size=259917 sha256=c9662eca9e4e70b1bd09d5d0896ebb3b0b226ed12886e555d03ea0466250cd0e
  Stored in directory: /root/.cache/pip/wheels/ff/48/ac/a77c79aa416ba6dd7bf487f2280b0471034f66141617965914
Successfully built gluonnlp
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tifffile 2020.9.3 has requirement numpy>=1.15.1, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: sea

In [ ]:
from bert_embedding import BertEmbedding

In [ ]:
embedding = BertEmbedding()

Vocab file is not found. Downloading.


In [ ]:
embedding("I am going to the park for playing with my friend")

[(['i'],
  [array([-3.25331599e-01, -3.46561611e-01,  4.29405212e-01, -3.96290034e-01,
           3.92348021e-02,  4.31209169e-02,  2.62787372e-01,  4.49371010e-01,
           8.42511803e-02, -7.80547619e-01, -7.68595338e-01,  1.82808898e-02,
           3.30514342e-01,  2.11116731e-01,  3.89601924e-02, -1.04872912e-01,
           6.49706542e-01,  3.01850110e-01, -8.97732675e-02,  6.37651980e-01,
           1.63249075e-02,  4.21480149e-01, -3.30491155e-01,  1.85020477e-01,
          -2.07559392e-02,  9.22393858e-01, -5.38871765e-01, -4.52164441e-01,
          -4.51066047e-01,  4.11571294e-01, -4.00682017e-02,  9.40975994e-02,
           4.08912599e-01,  3.52807462e-01, -5.01864612e-01, -5.43319404e-01,
          -1.77644752e-02,  1.02241397e-01, -6.06602311e-01, -2.47278795e-01,
           3.00695777e-01, -4.47480708e-01,  4.15057898e-01, -7.98903346e-01,
          -6.15273975e-02,  4.00952101e-01,  3.41679394e-01, -2.42737457e-01,
           3.38914990e-03, -6.01112694e-02, -6.85983956

In [ ]:
doc1 = nlp("Carbon is the 15th most abundant element in the Earth's crust, and the fourth most abundant element in the universe by mass after hydrogen, helium, and oxygen.")
doc2 = nlp("Carbon's abundance, its unique diversity of organic compounds, and its unusual ability to form polymers at the temperatures commonly encountered on Earth enables this element to serve as a common element of all known life.")

print(cosine_similarity(doc1.vector.reshape(1,96), doc2.vector.reshape(1,96)))

[[0.7655677]]


#Google Universal Encoder

In [ ]:
!pip3 install fasttext

     |████████████████████████████████| 71kB 1.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3019883 sha256=50be69d288cd3eef4d64f797a392498013f7fd7b9db02a7c5eafa952378147ac
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
def textrank_summarizer(text, num_sent):
  doc = nlp(text)
  sentence_list = []
  for sent in doc.sents:
    sentence_list.append(sent)
  sentences = []
  for sent in doc.sents:
    doc1 = nlp(str(sent))
    sentence = [str(token) for token in doc1]
    sentences.append(sentence)
  with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    embeddings = sess.run(embed(sentences[0]))
  embeddings = []
  with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    for sentence in sentences:
      embeddings.append(sess.run(embed(sentence)))
  average_embedding = []
  for embedding in embeddings:
    average = np.zeros(512)
    length = len(embeddings)
    for word_embedding in embedding:
      average = average+word_embedding
    average = average/length
    average_embedding.append(average)
  average_embedding = np.array(average_embedding)
  sim_mat = np.zeros([length, length])
  for i in range(length):
    for j in range(length):
      if i != j:
        sim_mat[i][j] = cosine_similarity(average_embedding[i].reshape(1,512), average_embedding[j].reshape(1,512))[0,0]
  nx_graph = nx.from_numpy_array(sim_mat)
  scores = nx.pagerank(nx_graph)
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentence_list)), reverse=True)
  final_summary = [ranked_sentences[i][1] for i in range(num_sent)]
  return final_summary

In [ ]:
textrank_summarizer(text, 10)

NameError: ignored

#Flask  ---end

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
app = Flask(__name__)
run_with_ngrok(app)   

@app.route('/', methods=['GET', 'POST'])
def test():
  data = request.json
  print(data['text'])
  return jsonify(data)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://102c86bfd772.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Oct/2020 09:43:55] "POST / HTTP/1.1" 200 -


hello world


In [ ]:
!pip install flask-ngrok

from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
app = Flask(__name__)
run_with_ngrok(app)   

@app.route("/", methods=['GET', 'POST'])
def home():
  data = request.json
  text = data['text']
  print(text)
  questions = []
  summary = text_summarizer(text)
  count = 1
  for sent in summary:
    ques, ans = return_question_with_answer(sent)
    if(ques!=-1):
      distractors = distractor_generator(ans)
      if(len(distractors)>0):
        distractors.append(ans)
        dictionary = {
            "_id": count,
            "question": ques,
            "correct": ans.lower(),
            "options": distractors
        }
        count = count+1
        questions.append(dictionary)
  print(questions)
  
  return jsonify({
      "questions": questions
  })
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b5f24b379edb.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
